In [30]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd

In [31]:
model = gb.Model("Can2Oil Transshipment Problem")

In [32]:
costs_df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Models & Applications/Assignment 1/Cost_Production_to_Refinement.csv')
costs_transshipment_df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Models & Applications/Assignment 1/Cost_Production_to_Transshipment.csv')
costs_transshipment_to_refinement_df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Models & Applications/Assignment 1/Cost_Transshipment_to_Refinement.csv')
capacity_transship_production_df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Models & Applications/Assignment 1/Capacity_for_Transship_Production_Facilities.csv')
demand_refinement_df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Models & Applications/Assignment 1/Refinement_Demand.csv')

print(costs_df.head())
print(costs_transshipment_df.head())
print(costs_transshipment_to_refinement_df.head())
print(capacity_transship_production_df.head())
print(demand_refinement_df.head())

   ProductionFacility  RefinementCenter      Cost
0                   1                 1  4.252733
1                   1                 2  4.567726
2                   1                 3  4.696484
3                   1                 4  2.678741
4                   1                 5  4.272451
   ProductionFacility  TransshipmentHub      Cost
0                   1                 1  2.378826
1                   1                 2  0.863842
2                   2                 1  1.666982
3                   2                 2  2.119488
4                   3                 1  2.174880
   TransshipmentHub  RefinementCenter      Cost
0                 1                 1  1.572329
1                 1                 2  3.465474
2                 1                 3  2.244062
3                 1                 4  3.773839
4                 1                 5  3.262652
   ProductionFacility  Capacity
0                   1       374
1                   2       444
2               

In [33]:
# First, we need to create tuples for the indices
production_facilities = costs_df['ProductionFacility'].unique()
refinement_centers = costs_df['RefinementCenter'].unique()
prod_refin_tuples = gb.tuplelist((p, r) for p in production_facilities for r in refinement_centers)

# Now we need to create a dictionary for the costs, keyed by these tuples
costs_dict = {(row['ProductionFacility'], row['RefinementCenter']): row['Cost'] for _, row in costs_df.iterrows()}


In [34]:
x = model.addVars(prod_refin_tuples, 
                  obj=costs_dict, 
                  lb=0, vtype=GRB.CONTINUOUS, 
                  name="Ship_Production_to_Refinement")

In [35]:
# Create tuples for the indices
production_facilities_transship = costs_transshipment_df['ProductionFacility'].unique()
transshipment_hubs = costs_transshipment_df['TransshipmentHub'].unique()
prod_trans_tuples = gb.tuplelist((p, t) for p in production_facilities_transship for t in transshipment_hubs)

# Create a dictionary for the costs, keyed by these tuples
trans_costs_dict = {(row['ProductionFacility'], row['TransshipmentHub']): row['Cost'] for _, row in costs_transshipment_df.iterrows()}

In [36]:
# Now we can define the decision variables for shipping from production facilities to transshipment centers
y = model.addVars(prod_trans_tuples, 
                  obj=trans_costs_dict, 
                  lb=0, vtype=GRB.CONTINUOUS, 
                  name="Ship_Production_to_Transshipment")

In [37]:
# Create tuples for the indices
transship_refin_tuples = gb.tuplelist(
    (t, r) for t in transshipment_hubs for r in refinement_centers
)

# Create a dictionary for the costs, keyed by these tuples
transship_to_refin_costs_dict = {
    (row['TransshipmentHub'], row['RefinementCenter']): row['Cost']
    for _, row in costs_transshipment_to_refinement_df.iterrows()
}

In [38]:
# Now we can define the decision variables for shipping from transshipment centers to refinement centers
z = model.addVars(transship_refin_tuples, 
                  obj=transship_to_refin_costs_dict, 
                  lb=0, vtype=GRB.CONTINUOUS, 
                  name="Ship_Transshipment_to_Refinement")

In [41]:
# Convert the capacity DataFrame to a dictionary for easier lookup
capacity_dict = capacity_transship_production_df.set_index('ProductionFacility')['Capacity'].to_dict()

# Update supply constraints for production facilities that ship to transshipment centers
for facility in production_facilities_transship:
    model.addConstr(
        gb.quicksum(y[facility, hub] for hub in transshipment_hubs)
        <= capacity_dict[facility],
        name=f"Supply_Capacity_{facility}"
    )


In [42]:
# Define the capacities for the transshipment centers as a dictionary
transship_capacities = {1: 1317, 2: 1453}

# Update transshipment constraints for transshipment centers
for hub in transshipment_hubs:
    model.addConstr(
        gb.quicksum(z[hub, center] for center in refinement_centers) 
        <= transship_capacities[hub],
        name=f"Transshipment_Capacity_{hub}"
    )


In [43]:
# Convert the demand DataFrame to a dictionary for easier lookup
demand_dict = demand_refinement_df.set_index('RefinementCenter')['Demand'].to_dict()

# Update demand constraints for refinement centers
for center in refinement_centers:
    model.addConstr(
        (gb.quicksum(x[facility, center] for facility in production_facilities if (facility, center) in x) +
         gb.quicksum(z[hub, center] for hub in transshipment_hubs if (hub, center) in z))
        >= demand_dict[center],
        name=f"Demand_{center}"
    )


In [44]:
# The objective function is already set through the 'obj' parameter in the decision variables

# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 38 rows, 165 columns and 210 nonzeros
Model fingerprint: 0x28d210c6
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+03]
Presolve removed 32 rows and 155 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1881492e+04   6.163136e+02   0.000000e+00      0s
       7    2.0891777e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.089177707e+04


In [45]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Number of Decision Variables:  165


In [46]:
# Value of the objective function
print("Total Transportation cost: ", model.objVal)

Total Transportation cost:  20891.77707233583


In [47]:
# Print the decision variables
for v in model.getVars():
    if v.x > 0:
        print(f"{v.varName}: {v.x}")

Ship_Production_to_Refinement[1,4]: 1838.0
Ship_Production_to_Refinement[2,2]: 295.0
Ship_Production_to_Refinement[8,5]: 1665.0
Ship_Production_to_Refinement[16,1]: 878.5
Ship_Production_to_Refinement[21,3]: 1940.0
Ship_Transshipment_to_Refinement[1,1]: 658.5
Ship_Transshipment_to_Refinement[2,2]: 1453.0
